## FEATS-sep
This tagger makes a separate model for each feature of the FEATS field. This exapmle is for demonstation purpose only. It take a lot of resourses for training and futher use, and the resulting performance we've gotten after training is lower than that of the `FeatsJointTagger`. Thus, in practical meaning this tagger is not very helpful. We're not adjusted any hyperparameters and just took them from already adjusted `FeatsJointTagger` class. They are surely can be improved, especially if you just pretrain them with this pipeline (running only stages 1 and 2) and then tune each produced model separately as the `FeatsTagger` object. Maybe, it worth to train and tune the model only for a few particular features, performance on which are higher than that of the `FeatsJointTagger`. But in our production pipeline we don't use it (mostly, because of the lack of resouces).
##### Warning: the training pipeline below is gonna be long!<br />

In [1]:
from corpuscula.corpus_utils import download_ud, UniversalDependencies, \
                                    AdjustedForSpeech
import junky
from mordl.feats_tagger import FeatsSeparateTagger

BERT_MODEL_FN = 'xlm-roberta-base'
MODEL_FN = 'feats-sep_model'
SEED=42
BERT_MAX_LEN, BERT_EPOCHS, BERT_BATCH_SIZE = 0, 3, 8
DEVICE='cuda:0'

corpus_name = 'UD_Russian-SynTagRus'
download_ud(corpus_name, overwrite=False)
corpus = UniversalDependencies(corpus_name)
#corpus = AdjustedForSpeech(corpus)

In [2]:
tagger = FeatsSeparateTagger()
tagger.load_train_corpus(corpus.train)
tagger.load_test_corpus(corpus.dev)

_ = tagger.train(MODEL_FN, feats=None, device=DEVICE, control_metric='accuracy',
                 max_epochs=None, min_epochs=0, bad_epochs=5,
                 max_grad_norm=None, tags_to_remove=None, word_emb_type='bert',
                 word_emb_path=BERT_MODEL_FN, word_transform_kwargs={
                     'max_len': BERT_MAX_LEN, 'hidden_ids': 10, 'aggregate_subtokens_op': 'absmax'
                     # BertDataset.transform() (for BERT-descendant models)
                     # params:
                     # {'max_len': 0, 'batch_size': 64, 'hidden_ids': '10',
                     #  'aggregate_hiddens_op': 'cat',
                     #  'aggregate_subtokens_op': 'absmax', 'to': junky.CPU,
                     #  'loglevel': 1}
                     # WordDataset.transform() (for other models) params:
                     # {'check_lower': True}
                 },
                 stage1_params=None,
                     # {'lr': .0001, 'betas': (0.9, 0.999), 'eps': 1e-8,
                     #  'weight_decay': 0, 'amsgrad': False,
                     #  'max_epochs': None, 'min_epochs': None,
                     #  'bad_epochs': None, 'batch_size': None,
                     #  'max_grad_norm': None}
                 stage2_params=None,
                     # {'lr': .001, 'momentum': .9, 'weight_decay': 0,
                     #  'dampening': 0, 'nesterov': False,
                     #  'max_epochs': None, 'min_epochs': None,
                     #  'bad_epochs': None, 'batch_size': None,
                     #  'max_grad_norm': None}
                 stage3_params={
                     'save_as': MODEL_FN.replace('-bert_model', '_' + BERT_MODEL_FN)
                              + f'_len{BERT_MAX_LEN}_ep{BERT_EPOCHS}_bat{BERT_BATCH_SIZE}_seed{SEED}',
                     'epochs': BERT_EPOCHS,
                     'batch_size': BERT_BATCH_SIZE,
                     'lr': 2e-5, 'num_warmup_steps': 3,
                     # {'save_as': None, 'max_epochs': 3, 'batch_size': 8,
                     #  'lr': 2e-5, 'betas': (0.9, 0.999), 'eps': 1e-8,
                     #  'weight_decay': .01, 'amsgrad': False,
                     #  'num_warmup_steps': 3, 'max_grad_norm': 1.}
                 },
                 stages=[1, 2, 3, 1, 2], save_stages=False,
                 learn_on_padding=True, remove_padding_intent=False,
                 seed=SEED, keep_embs=False,
                 rnn_emb_dim=None, cnn_emb_dim=200, cnn_kernels=range(1, 7),
                 upos_emb_dim=200, emb_bn=True, emb_do=.2,
                 final_emb_dim=512, pre_bn=True, pre_do=.5,
                 lstm_layers=1, lstm_do=0, tran_layers=0, tran_heads=8,
                 post_bn=True, post_do=.4)

Train: Load corpus
[=================================================] 48814           
Corpus has been loaded: 48814 sentences, 871526 tokens
Test: Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


###### FEATS TAGGER TRAINING PIPELINE ######

We're gonna train separate models for all FEATS in train corpus. Feats are:

Animacy, Aspect, Case, Degree, Foreign, Gender, Mood, Number, Person, Polarity, Tense, Variant, VerbForm, Voice


=== FEATS:Animacy TAGGER TRAINING PIPELINE ===

DATASETS CREATION
Tokenizing
100%|██████████| 48814/48814 [00:34<00:00, 1427.27it/s]
Vectorizing
100%|██████████| 763/763 [00:52<00:00, 14.40it/s]
Reordering
100%|██████████| 48814/48814 [01:32<00:00, 528.72it/s]
Tokenizing
100%|██████████| 6584/6584 [00:04<00:00, 1479.85it/s]
Vectorizing
100%|██████████| 103/103 [00:07<00:00, 13.64it/s]
Reordering
100%|██████████| 6584/6584 [00:12<00:00, 521.65it/s]

MODEL CREATION

MODEL TRAINING 1 (STAGE 1, SEED 2746317214)
Epoch 1: 100%|██████████| 48814/48814 [01:34<00:00, 517.67it/s, train_loss=0.0148]
Epoch 1: 
Losses: train = 0.03655637, test = 0.00974577
Test: accuracy = 0.99202451
Test: precision = 0.98343835
Test: recall = 0.98055032
Test: f1_score = 0.98197526


Epoch 26: 100%|██████████| 48814/48814 [01:34<00:00, 517.47it/s, train_loss=0.00167]
Epoch 26: 
Losses: train = 0.00158213, test = 0.00596033
Test: accuracy = 0.99640470
Test: precision = 0.98967681
Test: recall = 0.98954075
Test: f1_score = 0.98960748
BAD EPOCHS: 3 (<< <)
Epoch 27: 100%|██████████| 48814/48814 [01:34<00:00, 517.68it/s, train_loss=0.00143]
Epoch 27: 
Losses: train = 0.00149259, test = 0.00591342
Test: accuracy = 0.99651441
Test: precision = 0.98975257
Test: recall = 0.98971627
Test: f1_score = 0.98973442
new maximum score 0.99651441
Config saved
Saving state_dict... done.
Epoch 28: 100%|██████████| 48814/48814 [01:34<00:00, 518.48it/s, train_loss=0.00152]
Epoch 28: 
Losses: train = 0.00144995, test = 0.00617226
Test: accuracy = 0.99635406
Test: precision = 0.98966063
Test: recall = 0.98879548
Test: f1_score = 0.98922718
BAD EPOCHS: 1 (<< <)
Epoch 29: 100%|██████████| 48814/48814 [01:34<00:00, 516.65it/s, train_loss=0.00155]
Epoch 29: 
Losses: train = 0.00142819, test =

Epoch 17: 100%|██████████| 48814/48814 [01:32<00:00, 529.29it/s, train_loss=0.00108] 
Epoch 17: 
Losses: train = 0.00102871, test = 0.00595425
Test: accuracy = 0.99651441
Test: precision = 0.98960479
Test: recall = 0.98998333
Test: f1_score = 0.98979385
BAD EPOCHS: 5 (<< <)
Maximum bad epochs exceeded. Process has been stopped. Best score 0.996556613327932 (on epoch 10)
Elapsed time: 28m 13s

MODEL TRAINING 3 (STAGE 3, SEED 107420370)
Loading state_dict... done.
Epoch 1: 100%|██████████| 6102/6102 [19:43<00:00,  5.16it/s, train_loss_EMA=0.00463] 
Epoch 1: 
    Losses: train = 0.00563635, test = 0.00475066
    Test: accuracy = 0.99632030
    Test: precision = 0.99001535
    Test: recall = 0.98732896
    Test: f1_score = 0.98865942
BAD EPOCHS: 1 (<< <)
Epoch 2: 100%|██████████| 6102/6102 [19:42<00:00,  5.16it/s, train_loss_EMA=0.000284]
Epoch 2: 
    Losses: train = 0.00276315, test = 0.00490151
    Test: accuracy = 0.99663257
    Test: precision = 0.98814631
    Test: recall = 0.9913934



=== FEATS:Aspect TAGGER TRAINING PIPELINE ===

DATASETS CREATION
Tokenizing
100%|██████████| 48814/48814 [01:01<00:00, 795.22it/s]
Vectorizing
100%|██████████| 763/763 [00:52<00:00, 14.44it/s]
Reordering
100%|██████████| 48814/48814 [01:32<00:00, 530.41it/s]
Tokenizing
100%|██████████| 6584/6584 [00:04<00:00, 1449.86it/s]
Vectorizing
100%|██████████| 103/103 [00:07<00:00, 13.64it/s]
Reordering
100%|██████████| 6584/6584 [00:12<00:00, 520.02it/s]

MODEL CREATION

MODEL TRAINING 1 (STAGE 1, SEED 2746317214)
Epoch 1: 100%|██████████| 48814/48814 [01:34<00:00, 517.93it/s, train_loss=0.00389]
Epoch 1: 
Losses: train = 0.02102026, test = 0.00172315
Test: accuracy = 0.99870029
Test: precision = 0.99242006
Test: recall = 0.99239917
Test: f1_score = 0.99240957
new maximum score 0.99870029
Config saved
Saving state_dict... done.
Epoch 2: 100%|██████████| 48814/48814 [01:34<00:00, 518.70it/s, train_loss=0.00228]
Epoch 2: 
Losses: train = 0.00248412, test = 0.00152789
Test: accuracy = 0.99894504

...

Epoch 8: 100%|██████████| 48814/48814 [01:32<00:00, 529.28it/s, train_loss=6.6e-5] 
Epoch 8: 
Losses: train = 0.00005932, test = 0.00029605
Test: accuracy = 0.99987340
Test: precision = 0.99633490
Test: recall = 0.99735781
Test: f1_score = 0.99684519
BAD EPOCHS: 1 (<< >)
Epoch 9: 100%|██████████| 48814/48814 [01:33<00:00, 522.66it/s, train_loss=6.13e-5]
Epoch 9: 
Losses: train = 0.00006722, test = 0.00029338
Test: accuracy = 0.99987340
Test: precision = 0.99633490
Test: recall = 0.99735781
Test: f1_score = 0.99684519
BAD EPOCHS: 2 (<< =)
Epoch 10: 100%|██████████| 48814/48814 [01:32<00:00, 527.98it/s, train_loss=4.56e-5]
Epoch 10: 
Losses: train = 0.00005748, test = 0.00030844
Test: accuracy = 0.99987340
Test: precision = 0.99633490
Test: recall = 0.99735781
Test: f1_score = 0.99684519
BAD EPOCHS: 3 (<< =)
Epoch 11: 100%|██████████| 48814/48814 [01:33<00:00, 523.43it/s, train_loss=3.75e-5] 
Epoch 11: 
Losses: train = 0.00005584, test = 0.00030615
Test: accuracy = 0.99986497
Test: preci

Loading state_dict... done.
Epoch 1: 100%|██████████| 6102/6102 [20:26<00:00,  4.97it/s, train_loss_EMA=4.09e-5] 
Epoch 1: 
    Losses: train = 0.00106393, test = 0.00008131
    Test: accuracy = 0.99991560
    Test: precision = 0.99710741
    Test: recall = 0.99963894
    Test: f1_score = 0.99836987
BAD EPOCHS: 1 (<< <)
Epoch 2: 100%|██████████| 6102/6102 [20:25<00:00,  4.98it/s, train_loss_EMA=0.00788] 
Epoch 2: 
    Losses: train = 0.00027843, test = 0.00006137
    Test: accuracy = 0.99995780
    Test: precision = 0.99870789
    Test: recall = 0.99966032
    Test: f1_score = 0.99918364
new maximum score 0.99995780
Saving checkpoint to ('feats-variant_feats-sep_model_len0_ep3_bat8_seed42', 'feats-sep_model-variant')
Config saved
Saving state_dict... done.
Epoch 3: 100%|██████████| 6102/6102 [20:26<00:00,  4.98it/s, train_loss_EMA=8.3e-6]  
Epoch 3: 
    Losses: train = 0.00013120, test = 0.00006312
    Test: accuracy = 0.99996624
    Test: precision = 0.99902923
    Test: recall = 0.9

Epoch 4: 100%|██████████| 48814/48814 [01:32<00:00, 525.71it/s, train_loss=1.52e-5]
Epoch 4: 
Losses: train = 0.00002147, test = 0.00002370
Test: accuracy = 0.99998312
Test: precision = 0.99980892
Test: recall = 0.99952815
Test: f1_score = 0.99966826
BAD EPOCHS: 4 (<< =)
Epoch 5: 100%|██████████| 48814/48814 [01:32<00:00, 528.97it/s, train_loss=9.87e-6]
Epoch 5: 
Losses: train = 0.00001663, test = 0.00002730
Test: accuracy = 0.99998312
Test: precision = 0.99980892
Test: recall = 0.99952815
Test: f1_score = 0.99966826
BAD EPOCHS: 5 (<< =)
Maximum bad epochs exceeded. Process has been stopped. No models could surpass `best_score=0.9999915603267842` given
Elapsed time: 8m 20s

=== FEATS:VerbForm TAGGER TRAINING HAS FINISHED === Total time: 2h 1m 9s ===

Use the `.load('feats-sep_model-verbform')` method to start working with the FEATS:VerbForm tagger.


=== FEATS:Voice TAGGER TRAINING PIPELINE ===

DATASETS CREATION
Tokenizing
100%|██████████| 48814/48814 [00:56<00:00, 857.20it/s]
Vectori

Epoch 25: 100%|██████████| 48814/48814 [01:33<00:00, 520.47it/s, train_loss=0.000557]
Epoch 25: 
Losses: train = 0.00056172, test = 0.00153738
Test: accuracy = 0.99901256
Test: precision = 0.98730346
Test: recall = 0.98451562
Test: f1_score = 0.98583598
BAD EPOCHS: 2 (<< >)
Epoch 26: 100%|██████████| 48814/48814 [01:34<00:00, 516.50it/s, train_loss=0.000575]
Epoch 26: 
Losses: train = 0.00054157, test = 0.00143729
Test: accuracy = 0.99902944
Test: precision = 0.98749066
Test: recall = 0.98475102
Test: f1_score = 0.98605412
BAD EPOCHS: 2 (<< >)
Epoch 27: 100%|██████████| 48814/48814 [01:34<00:00, 517.48it/s, train_loss=0.000556]
Epoch 27: 
Losses: train = 0.00051208, test = 0.00143793
Test: accuracy = 0.99903788
Test: precision = 0.98679023
Test: recall = 0.98566945
Test: f1_score = 0.98622550
BAD EPOCHS: 2 (<< >)
Epoch 28: 100%|██████████| 48814/48814 [01:35<00:00, 512.98it/s, train_loss=0.00056] 
Epoch 28: 
Losses: train = 0.00051398, test = 0.00136847
Test: accuracy = 0.99912227
Test

Epoch 7: 100%|██████████| 48814/48814 [01:34<00:00, 514.21it/s, train_loss=0.000211]
Epoch 7: 
Losses: train = 0.00024817, test = 0.00166035
Test: accuracy = 0.99890284
Test: precision = 0.98423486
Test: recall = 0.98392739
Test: f1_score = 0.98407943
BAD EPOCHS: 5 (<< <)
Maximum bad epochs exceeded. Process has been stopped. No models could surpass `best_score=0.9991222739855513` given
Elapsed time: 11m 56s

MODEL TRAINING 5 (STAGE 2, SEED 1181241944)
Loading state_dict... done.
Epoch 1: 100%|██████████| 48814/48814 [01:32<00:00, 526.22it/s, train_loss=0.00033] 
Epoch 1: 
Losses: train = 0.00032482, test = 0.00139186
Test: accuracy = 0.99898724
Test: precision = 0.98615768
Test: recall = 0.98460877
Test: f1_score = 0.98535811
BAD EPOCHS: 1 (<< <)
Epoch 2: 100%|██████████| 48814/48814 [01:32<00:00, 526.30it/s, train_loss=0.000359]
Epoch 2: 
Losses: train = 0.00032785, test = 0.00142960
Test: accuracy = 0.99900412
Test: precision = 0.98646681
Test: recall = 0.98480175
Test: f1_score = 0

We don't have enough resouces to load all the models simultaneously, so we just applied them separatedly. If you have, you can execute 3 cells above and not run 3 cells below.

In [3]:
from copy import deepcopy
from corpuscula import Conllu
import json
import mordl

In [4]:
res_dev = 'corpora/_dev_' + MODEL_FN.replace('-sep_model', '.conllu')
res_test = 'corpora/_test_' + MODEL_FN.replace('-sep_model', '.conllu')
dev = deepcopy(corpus.dev)
test = deepcopy(corpus.test)
with open(MODEL_FN + mordl.defaults.CONFIG_EXT, 'rt') as f:
    config = json.load(f)
junky.clear_tqdm()

In [5]:
for feat, path in sorted(config.items()):
    tagger = mordl.FeatTagger(f'FEATS:{feat}')
    tagger.load(path)
    dev = list(tagger.predict(dev, clone_ds=True))
    test = list(tagger.predict(test, clone_ds=True))
Conllu.save(dev, res_dev)
Conllu.save(test, res_test)
tagger = FeatsSeparateTagger()

Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.


Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


Vectorizing
100%|██████████| 103/103 [00:07<00:00, 13.82it/s]
Reordering
100%|██████████| 6584/6584 [00:12<00:00, 521.31it/s]


Load corpus
[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens


Vectorizing
100%|██████████| 102/102 [00:07<00:00, 14.03it/s]
Reordering
100%|██████████| 6491/6491 [00:12<00:00, 518.23it/s]
Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.
Vectorizing
100%|██████████| 103/103 [00:07<00:00, 13.87it/s]
Reordering
100%|██████████| 6584/6584 [00:12<00:00, 523.81it/s]
Vectorizing
100%|██████████| 102/102 [00:07<00:00, 14.04it/s]
Reordering
100%|██████████| 6491/6491 [00:12<00:00, 526.70it/s]
Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.
Vectorizing
100%|██████████| 103/103 [00:07<00:00, 13.81it/s]
Reordering
100%|██████████| 6584/6584 [00:12<00:00, 519.51it/s]
Vectorizing
100%|██████████| 102/102 [00:07<00:00, 13.98it/s]
Reordering
100%|██████████| 6491/6491 [00:12<00:00, 517.04it/s]
Loading dataset... done.
Creating model... done.
Loading state_dict... done.
Fit corpus dict... done.
Vectorizing
100%|██████████| 103/103 [00:07<00:00, 13.86it/s]
Re

Save corpus
[=======] 6584                                                     
Corpus has been saved
Save corpus
[=======] 6491                                                     
Corpus has been saved


### Evaluation

In [6]:
_ = tagger.evaluate(corpus.dev, res_dev)

Load corpus
[======> 6500                                                      

Evaluating FEATS
Load corpus
[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens
FEATS total: 118488 tokens, 287009 tags
    correct: 116981 tokens, 285060 tags
      wrong: 1507 tokens, 1949 tags [212 excess / 285 absent / 1452 wrong type]
   Accuracy: 0.9793230245736317 / 0.9932092721831023
[Total accuracy: 0.9872814124637094 / 0.9932092721831023]
[By sentence accuracy: 0.8280680437424058]


[=======] 6584                                                     
Corpus has been loaded: 6584 sentences, 118692 tokens


In [7]:
_ = tagger.evaluate(corpus.test, res_test)

Evaluating FEATS
Load corpus
[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens
FEATS total: 117329 tokens, 284932 tags
    correct: 115714 tokens, 282776 tags
      wrong: 1615 tokens, 2156 tags [279 excess / 364 absent / 1513 wrong type]
   Accuracy: 0.9776572638103012 / 0.9924332823270113
[Total accuracy: 0.9862352870986713 / 0.9924332823270113]
[By sentence accuracy: 0.8129718071175474]


Load corpus
[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens


In [8]:
corp_gold = list(corpus.test())
corp_test = list(tagger._get_corpus(res_test))
tags = sorted(set(x for x in corp_gold
                    for x in x[0]
                    for x in x['FEATS'].keys()))
for tag in tags:
    print('{}: {}'.format(
        tag, tagger.evaluate(corp_gold, corp_test,
                             feats=tag, log_file=None)
    ))

Load corpus
[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens


Load corpus
[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens
Animacy: 0.989450524653052
Aspect: 0.9922038547607016
Case: 0.989055773462164
Degree: 0.9983400521697889
Foreign: 0.7720364741641338
Gender: 0.9891674531205121
Mood: 0.9983397897066962
Number: 0.9954938657369339
Person: 0.9978638564511535
Polarity: 0.8444444444444444
Tense: 0.9975888551527058
Variant: 0.9962756052141527
VerbForm: 0.9994946942900454
Voice: 0.9913376164007797


As we can see, the overall performance is notable lower than the performance of the `FeatJointTagger`. However, a number of features are detected slightly better with separate tagger. Here, it worth reminding that we didn't adjust any hyperparameter, and we hope that this performance can be improved.

For comparison, we put here accuracy values produced by `FeatsJointTagger` one the *SynTagRus* test corpus:

Animacy: 0.9899892843043258<br />
Aspect: 0.9945146156622158<br />
Case: 0.9884052425141108<br />
Degree: 0.9979249421948183<br />
Foreign: 0.7455089820359282<br />
Gender: 0.9891480501364186<br />
Mood: 0.9987825124515772<br />
Number: 0.9954606009769577<br />
Person: 0.9994301994301994<br />
Polarity: 0.9555555555555556<br />
Tense: 0.9980355388874007<br />
Variant: 0.9981366459627329<br />
VerbForm: 0.9996391194514616<br />
Voice: 0.991699747383616

We can't find a correlation between the cases when separate taggers prevail on joint ones. In the cell below we list the number of tokens for each separate feature. But there are no dependence with it, too.

In [9]:
num_tags = {tag: 0 for tag in sorted(tags)}
for sent in corpus.test():
    for tok in sent[0]:
        for feat in tok['FEATS']:
            num_tags[feat] += 1
for tag, num in sorted(num_tags.items()):
    print(f'{tag}: {num}')

Load corpus
[=====> 5800                                                       

Animacy: 35357
Aspect: 13853
Case: 52187
Degree: 16866
Foreign: 309
Gender: 48666
Mood: 9031
Number: 60726
Person: 7019
Polarity: 45
Tense: 11194
Variant: 1609
VerbForm: 13853
Voice: 13853


[=======] 6491                                                     
Corpus has been loaded: 6491 sentences, 117523 tokens
